In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import datetime
import time
train = pd.read_csv('train.csv')
train.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,87540,12,1,13,497,2017-11-07 09:30:38,NaN,0
1,105560,25,1,17,259,2017-11-07 13:40:27,NaN,0
2,101424,12,1,19,212,2017-11-07 18:05:24,NaN,0
3,94584,13,1,13,477,2017-11-07 04:58:08,NaN,0
4,68413,12,1,1,178,2017-11-09 09:00:09,NaN,0


In [2]:
def convertToEpochTime(s):
    if s == '-1':
        return -1
    gDate = datetime.datetime.strptime(s,"%Y-%m-%d %H:%M:%S")
    #eTime = (gDate - datetime(1970, 1, 1)).total_seconds()
    epoch = time.mktime(gDate.timetuple())
    return epoch

In [3]:
train['click_time'] = train['click_time'].apply(lambda c:convertToEpochTime(c))

In [4]:
train.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,87540,12,1,13,497,1510027238,NaN,0
1,105560,25,1,17,259,1510042227,NaN,0
2,101424,12,1,19,212,1510058124,NaN,0
3,94584,13,1,13,477,1510010888,NaN,0
4,68413,12,1,1,178,1510198209,NaN,0


In [5]:
train['attributed_time'] = train['attributed_time'].fillna('-1')

In [6]:
train.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,87540,12,1,13,497,1510027238,-1,0
1,105560,25,1,17,259,1510042227,-1,0
2,101424,12,1,19,212,1510058124,-1,0
3,94584,13,1,13,477,1510010888,-1,0
4,68413,12,1,1,178,1510198209,-1,0


In [7]:
train['attributed_time'] = train['attributed_time'].apply(lambda c:convertToEpochTime(c))

In [8]:
train.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,87540,12,1,13,497,1510027238,-1,0
1,105560,25,1,17,259,1510042227,-1,0
2,101424,12,1,19,212,1510058124,-1,0
3,94584,13,1,13,477,1510010888,-1,0
4,68413,12,1,1,178,1510198209,-1,0


In [9]:
train_output = train['is_attributed']

In [10]:
train_output.head()


0    0
1    0
2    0
3    0
4    0
Name: is_attributed, dtype: int64

In [11]:
drop_feature = ['attributed_time','is_attributed']

In [12]:
train = train.drop(drop_feature,axis = 1 )

In [13]:
train.head()

,ip,app,device,os,channel,click_time
0,87540,12,1,13,497,1510027238
1,105560,25,1,17,259,1510042227
2,101424,12,1,19,212,1510058124
3,94584,13,1,13,477,1510010888
4,68413,12,1,1,178,1510198209


In [14]:
test = pd.read_csv('test.csv')

In [15]:
test.head()

,click_id,ip,app,device,os,channel,click_time
0,0,5744,9,1,3,107,2017-11-10 04:00:00
1,1,119901,9,1,3,466,2017-11-10 04:00:00
2,2,72287,21,1,19,128,2017-11-10 04:00:00
3,3,78477,15,1,13,111,2017-11-10 04:00:00
4,4,123080,12,1,13,328,2017-11-10 04:00:00


In [16]:
test['click_time'] = test['click_time'].apply(lambda c:convertToEpochTime(c))
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18790469 entries, 0 to 18790468
Data columns (total 7 columns):
click_id      int64
ip            int64
app           int64
device        int64
os            int64
channel       int64
click_time    float64
dtypes: float64(1), int64(6)
memory usage: 1.1 GB


In [17]:
y = test
y = y.drop(['click_id'],axis = 1 )

In [18]:
from sklearn import tree
model = tree.DecisionTreeClassifier(criterion='gini')
model.fit(train,train_output)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [19]:
model.score(train,train_output)

1.0

In [21]:
prediction = model.predict(y)

In [22]:
my_submission = pd.DataFrame({'click_id': test.click_id, 'is_attributed': prediction})
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)

In [23]:
ans = pd.read_csv('submission.csv')

In [24]:
ans.head()

,click_id,is_attributed
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [25]:
ans.is_attributed.value_counts()

0    18773175
1       17294
Name: is_attributed, dtype: int64